In [1]:
from huggingface_hub import InferenceClient
import json
import random
from tqdm import tqdm
import pandas as pd
import os

c:\ProgramData\anaconda3\envs\bmw_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


First we need to make sure you are being in the right folder. We assume you are in Downloads -> meme_caption_generator; if not, please adjust the path.

In [5]:
# Your home directry:
home_dir = os.path.expanduser("~")
home_dir = os.path.join(home_dir, 'Downloads', "meme_caption_generator")

In [6]:
home_dir

'C:\\Users\\Nursulu_1\\Downloads\\meme_caption_generator'

We also will be using HuggingFace token for InferenceAPI.

In [29]:
hf_token = "hf_YOUR_TOKEN"

I will use LLaMA3-8B-Instruct to label my images with topic

In [21]:
with open(os.path.join(home_dir, "memes_900k_files", "y_u_memes.json"), 'r') as file:
    y_u_memes = json.load(file)

- Remove non-english texts
- Remove the "< sep >" column

In [19]:
from langdetect import detect

In [20]:
def preprocess_text(text):
    # detect language. If not english, remove
    lan = detect(text)
    if lan != 'en':
        return None
    text = text.replace("<sep> ", "")
    return text

In [22]:
#before processing
len(y_u_memes)

6000

In [23]:
y_u_memes = list(map(preprocess_text, y_u_memes))
random.shuffle(y_u_memes)
y_u_memes = [el for el in y_u_memes if el != None]
len(y_u_memes)

3132

In [187]:
# Take a subset for training
y_u_memes = y_u_memes[:1000]

In [188]:
for i in range(len(y_u_memes)):
    y_u_memes[i] = y_u_memes[i].replace("<emp>", "")

Generate examples for the model to understand the format

In [24]:
y_u_memes_examples = y_u_memes[-10:]
y_u_memes_examples

['Mayans Y u no finish calendar?',
 'Y u so strict!!! Cannot even smash cake into Chers face',
 'FLAVOURED BUTTER Y U NO EXIST?',
 'bullies y u no see that harassement not funny?',
 'to all parents y u no understand we cannot pause online games',
 'Y U Bring knife to gunfight?',
 'shabbirv, y u no stop doing memes atleast for 1 minute?',
 'Stewie y u no try to kill lois anymore',
 'yu want me have vasectomy? Y U No like sperm?',
 'QUDRAT WHY U SO HANDSOME?!']

Describe manually topics for the 10 selected memes. Below is an example of how I did it (for other samples)

In [26]:
topics = ['a person using a bad internet browser', 'a sportsman who lost a competition', 'a person laying on a bad',
          'a researcher', 'a chubby man', 'a woman in warm clothes',
          'a gamer playing at the computer', 'a teenager using Iphone', 'a group of wild animals', 'a hairdresser at work',
          ]

In [27]:
# topics = ['a group of friends sitting together', 'a group of teenagers at the party', 'a couple',
#           'a couple trying to smell some food', 'a man looking in the camera and smiling', 'an old man wearing glasses',
#           'a man and a woman drinking coffee and smiling', 'a group of women checking out a guy on a street', 'a mother telling of his child', 'a couple talking to each other',
#           ]


# topics = ["competitions", "name and year", "new year celebration", "nature waking up", "game of thrones actor", "world war 3", "posting new memes", "release of gta5 game", "school shooting incident at sandy hook elementary school", "being angry at people unproficient in politics"]

In [28]:
len(topics)

10

In [204]:
examples_annotated = []
for i in range(len(y_u_memes_examples)):
    examples_annotated.append("Caption: " + y_u_memes_examples[i] + " Topic: " + topics[i])

This is how annotated examples might look:

In [205]:
examples_annotated

['Caption: Internet Explorer Y U no be good? Topic: a person using a bad internet browser',
 'Caption: LeBron james y u no win championship? Topic: a sportsman who lost a competition',
 'Caption: bed is comfy in the morning why u no comfy at night? Topic: a person laying on a bad',
 'Caption: OPeratins research Y U So difficult?! Topic: a researcher',
 'Caption: miserableatbest y u so fat? Topic: a chubby man',
 'Caption: CALIFORNIA Y U SO COLD I GOTTA VACATION IN LAS VEGAS!!! Topic: a woman in warm clothes',
 'Caption: online game y u no lag for them? Topic: a gamer playing at the computer',
 'Caption: iPhone Y U no SIRI! Topic: a teenager using Iphone',
 'Caption: Y U HAVE CLAW? WE DNT LYK THE CLAW Topic: a group of wild animals',
 'Caption: Hair Trimmer y u no trim while charging? Topic: a hairdresser at work']

In [206]:
content = '''Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n

### Instruction:
  {}

  ### Input:
  {}

  ### Response:
  {}
  '''

In [207]:
caption_to_topic = []

In [209]:
instruction = "You are given a meme caption. Your task is to detect what topic is described in the caption. Only output the topic and nothing more.\n"
client = InferenceClient(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    token=hf_token,
)

for i in tqdm(range(1, 990)):
    caption = y_u_memes[i]
    # print(caption)
    full_response = ''

    for message in client.chat_completion(
        messages=[{"role": "user", "content": content.format(
        instruction, # instruction
        examples_annotated,
        f"Caption: {caption} Topic: ")}],
        max_tokens=30,
        stream=True,
        temperature=0.7,
        top_p=0.95 
    ):
        response = message.choices[0].delta.content
        for chunk in response:
            full_response += chunk
    # print(full_response)
    caption_to_topic.append((caption, full_response))
    with open('caption_to_topic_angry.json', 'w') as file:
        json.dump(caption_to_topic, file)

 30%|███       | 299/989 [02:30<05:47,  1.99it/s]


HfHubHTTPError: 429 Client Error: Too Many Requests for url: https://api-inference.huggingface.co/models/meta-llama/Meta-Llama-3-8B-Instruct/v1/chat/completions (Request ID: lcFjHi0hlTv-s44SerPRb)

Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate

In [150]:
caption_to_topic[-1]

('this face  will get you laid', 'a man looking in the camera and smiling')

In [33]:
len(caption_to_topic)

307

In [210]:
for example in examples_annotated:
    topic = example.split("Topic: ")[-1] 
    print(topic)
    txt = example.split("Caption: ")
    txt = txt[1].split(" Topic:")
    # print(txt)
    caption = txt[0]
    print(caption)
    caption_to_topic.append((caption, topic))

a person using a bad internet browser
Internet Explorer Y U no be good?
a sportsman who lost a competition
LeBron james y u no win championship?
a person laying on a bad
bed is comfy in the morning why u no comfy at night?
a researcher
OPeratins research Y U So difficult?!
a chubby man
miserableatbest y u so fat?
a woman in warm clothes
CALIFORNIA Y U SO COLD I GOTTA VACATION IN LAS VEGAS!!!
a gamer playing at the computer
online game y u no lag for them?
a teenager using Iphone
iPhone Y U no SIRI!
a group of wild animals
Y U HAVE CLAW? WE DNT LYK THE CLAW
a hairdresser at work
Hair Trimmer y u no trim while charging?


In [84]:
caption_to_topic[-1]

('brace yourselves <sep> everyone on facebok is about to become an expert in political science',
 'being angry at people unproficient in politics')

Remove the word "Topic:"

In [211]:
for i in range(len(caption_to_topic)):
    caption, topic = caption_to_topic[i]
    if "Topic: " in topic:
        topic = topic.replace("Topic: ", "")
        caption_to_topic[i] = (caption, topic)

In [215]:
len(caption_to_topic)

309

Sometimes memes are very offensive and the model refused to respond. Remove the examples that are offensive according to the model.

In [213]:
indices_to_remove = []
for i in range(len(caption_to_topic)):
    caption, topic = caption_to_topic[i]
    if "I cannot provide a response" in topic:
        indices_to_remove.append(i)


In [214]:
for el in indices_to_remove:
    caption_to_topic.pop(el)

In [216]:
with open('caption_to_topic_angry.json', 'w') as file:
    json.dump(caption_to_topic, file)

Create a dataframe to prepare a dataset

In [217]:
import pandas as pd

In [228]:
df_angry = pd.DataFrame()
df_angry['topic'] = [topic for caption, topic in caption_to_topic]
df_angry['caption'] = [caption for caption, topic in caption_to_topic]

In [229]:
df_angry

,topic,caption
0,a person asking a rhetorical question about so...,NICK GIANELLA Y U ONLY LIKE CARS N GUNS?!
1,musician,Germy why u no listen to other bands!!!
2,A person using a bad reasoning process.,Heart! Y U NO logical?
3,a person,women y u no order ur own fries?
4,a sports fan,Purdue Fans Y U NO have no banners?
...,...,...
620,a person wanting their post retweeted,@ASOT550 y u no retweet my post ?
621,a person not believing the signs,Y U NO Believe the signs??
622,a person being late,Jordan Bradley Y U ALWAYS SO LATE
623,a student wanting school windows to be opened,NPHS windows y u no open


In [230]:
df_angry.to_csv("df_angry.csv", index=None)

In [46]:
# with open('caption_to_topic_y_u_meme.json', 'w') as file:
#     json.dump(caption_to_topic, file)